In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data Cleaning and Preparation

In [2]:
# Read the CGPA dataset
df1 = pd.read_excel("CGPA.xlsx")

# Read the BITSAT information dataset
df2 = pd.read_excel("BITSAT-2018.xlsx").dropna()


# Clean out the 2019 batch data
df1["first4"] = df1["Campus ID"].apply(lambda x:x[:4])
df1 = df1[df1['first4'] == "2018"]

# Clean out higher degree data
df1 = df1[~df1["Campus ID"].str.contains("HS")]

# Join the two datasets based on the last 4 digits of ID number
df1["last4"] = df1["Campus ID"].apply(lambda x:x[-5:-1])
df2["last4"] = df2["ID No."].apply(lambda x:x[-5:-1])
df = pd.concat([df1.set_index("last4"), df2.set_index("last4")], axis=1, join='inner')
df.reset_index(inplace=True)

# Filter out and rename useful columns
df = df[["ID","Campus ID", "ID No.", "Name", "Gender", "Board", "CGPA", "BITSAT Score"]]
df.rename(columns={"Campus ID":"Final_ID","ID No.":"BITSAT_ID"}, inplace=True)
df.head()

,ID,Final_ID,BITSAT_ID,Name,Gender,Board,CGPA,BITSAT Score
0,41120180003,2018A1PS0003H,2018A1PS0003H,RAHUL PRABHAKAR,MALE,Central Board of Secondary Education - All Ind...,6.41,275
1,41120180006,2018A1PS0006H,2018A1PS0006H,VARUN HEMANT KUMAR PATEL,MALE,"Board of Intermediate Education, Telangana - I...",7.34,266
2,41120180007,2018A1PS0007H,2018A1PS0007H,HARSH SHARMA,MALE,Central Board of Secondary Education - All Ind...,6.94,267
3,41120180010,2018A1PS0010H,2018A1PS0010H,PRANAV ROY,MALE,Central Board of Secondary Education - All Ind...,8.76,283
4,41120180011,2018A1PS0011H,2018A1PS0011H,CHENNA SHREE PRIYA,FEMALE,"Board of Intermediate Education, Telangana - I...",8.33,286


In [3]:
# Function to map branch code to branch name
def branch(x):
    branchmap = {
        "A1":"BE Chemical",
        "A2":"BE Civil",
        "A3":"BE EEE",
        "A4":"BE Mechanical",
        "A5":"BE Pharma",
        "A7":"BE Computer Science",
        "A8":"BE ENI",
        "AA":"BE ECE",
        "B1":"MSc Biology",
        "B2":"MSc Chemistry",
        "B3":"MSc Economics",
        "B4":"MSc Mathematics",
        "B5":"MSc Physics"
    }
    return branchmap[x[4:6]]

# Function to check whether student is Single degree or Dual Degree
def dualite(x):
    if x[4:6] in ["B1","B2","B3","B4","B5"]:
        return "Dual Degree"
    else:
        return "Single Degree"
    
# Function to check whether student converted to thesis
def PStoTS(finalid,bitsat):
    if bitsat[6:8] == "PS" and finalid[6:8] == "TS":
        return True
    return False

# Add separate column using each function above
df["Branch"] = df["BITSAT_ID"].apply(branch)
df["Dualite"] = df["BITSAT_ID"].apply(dualite)
df["Board"] = df["Board"].apply(lambda x:x.split(" - ")[0])
df['ConvertedToTS'] = df.apply(lambda x: PStoTS(x.Final_ID, x.BITSAT_ID), axis=1)

## Data Visualisation

In [4]:
# Function to plot male vs female and single vs dual degree pie charts
def basic_data(df):
    # Create 2 subplots, one for each pie chart
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Male and Female Population Distribution in 2018 Batch", "Single and Dual Degree Population Distribution in 2018 Batch"), specs=[[{'type': 'domain'},{'type': 'domain'}]])
    
    # Add trace for male vs female graph
    fig.add_trace(go.Pie(labels=df['Gender'].value_counts().index,
                     values=df['Gender'].value_counts(),
                     legendgroup='MaleFemale',
                     legendgrouptitle=dict(text='Male vs Female'),
                    ),
              row=1, col=1)
    
    # Add trace for single vs dual degree graph
    fig.add_trace(go.Pie(labels=df['Dualite'].value_counts().index,
                     values=df['Dualite'].value_counts(),
                     legendgroup='SingleDual',
                     legendgrouptitle=dict(text='Single vs Dual degree')),
              row=1, col=2)
    
    # Custom hover template
    fig.update_traces(hovertemplate ='%{label}'+'<br>Number of students: %{value}<br><extra></extra>')
    fig.show()
basic_data(df)


In [5]:
# Function to plot branch population distribution
def students_per_branch(df):
    fig = px.pie(df,names=df.Branch)
    
    # Custom hover template and title
    fig.update_traces(hovertemplate ='Branch: %{label}'+'<br>Number of students: %{value}<br><extra></extra>')
    fig.update_layout(title="Number of students per Branch")
    fig.show()
students_per_branch(df)

In [6]:
def cgpa_per_branch(df):
    
    # Histogram to plot average CGPA per branch per gender
    fig = px.histogram(df, histfunc="avg", x="Branch", y="CGPA",
                color='Gender', barmode='group')
    fig.update_layout(title="Average CGPA per branch comparisons between Male and Female", xaxis_title='Branch', yaxis_title='CGPA')
    fig.show()
cgpa_per_branch(df)

In [7]:
# Function to plot histogram of number of students per CGPA bracket, with branchwise dropdown
def cghistogram(df):
    
    # Assign each student thier respective CGPA bin using pd.cut()
    df["label"] = pd.cut(df["CGPA"], bins=[0,1,2,3,4,5,6,7,8,9,10])
    df = df.astype({"label":"str"})

    fig = go.Figure()

    # Add a seprate trace for each branch's histogram
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE Chemical"].ID,x=df[df["Branch"]=="BE Chemical"].label,name="BE Chemical"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE Civil"].ID,x=df[df["Branch"]=="BE Civil"].label,name="BE Civil"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE EEE"].ID,x=df[df["Branch"]=="BE EEE"].label,name="BE EEE"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE Mechanical"].ID,x=df[df["Branch"]=="BE Mechanical"].label,name="BE Mechanical"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE Pharma"].ID,x=df[df["Branch"]=="BE Pharma"].label,name="BE Pharma"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE Computer Science"].ID,x=df[df["Branch"]=="BE Computer Science"].label,name="BE Computer Science"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE ENI"].ID,x=df[df["Branch"]=="BE ENI"].label,name="BE ENI"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="BE ECE"].ID,x=df[df["Branch"]=="BE ECE"].label,name="BE ECE"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="MSc Biology"].ID,x=df[df["Branch"]=="MSc Biology"].label,name="MSc Biology"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="MSc Chemistry"].ID,x=df[df["Branch"]=="MSc Chemistry"].label,name="MSc Chemistry"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="MSc Economics"].ID,x=df[df["Branch"]=="MSc Economics"].label,name="MSc Economics"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="MSc Mathematics"].ID,x=df[df["Branch"]=="MSc Mathematics"].label,name="MSc Mathematics"))
    fig.add_trace(go.Histogram(histfunc="count",y=df[df["Branch"]=="MSc Physics"].ID,x=df[df["Branch"]=="MSc Physics"].label,name="MSc Physics"))

    # Add buttons and configure dropdown
    fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All Branches",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True, True, True, True, True, True, True, True]},
                           {"title": "CGPA Ranges vs Number of students for All Branches",
                            "annotations": []}]),
                dict(label="BE Chemical",
                     method="update",
                     args=[{"visible": [True, False, False, False, False, False, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE Chemical",
                            "annotations": []}]),
                dict(label="BE Civil",
                     method="update",
                     args=[{"visible": [False, True, False, False, False, False, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE Civil",
                            "annotations": []}]),
                dict(label="BE EEE",
                     method="update",
                     args=[{"visible": [False, False, True, False, False, False, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE EEE",
                            "annotations": []}]),
                dict(label="BE Mechanical",
                     method="update",
                     args=[{"visible": [False, False, False, True, False, False, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE Mechanical",
                            "annotations": []}]),
                dict(label="BE Pharma",
                     method="update",
                     args=[{"visible": [False, False, False, False, True, False, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE Pharma",
                            "annotations": []}]),
                dict(label="BE Computer Science",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, True, False, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE Computer Science",
                            "annotations": []}]),
                dict(label="BE ENI",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, True, False, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE ENI",
                            "annotations": []}]),
                dict(label="BE ECE",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, True, False, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for BE ECE",
                            "annotations": []}]),
                dict(label="MSc Biology",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, True, False, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for MSc Biology",
                            "annotations": []}]),
                dict(label="MSc Chemistry",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, False, True, False, False, False]},
                           {"title": "CGPA Ranges vs Number of students for MSc Chemistry",
                            "annotations": []}]),
                dict(label="MSc Economics",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, False, False, True, False, False]},
                           {"title": "CGPA Ranges vs Number of students for MSc Economics",
                            "annotations": []}]),
                dict(label="MSc Mathematics",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, False, False, False, True, False]},
                           {"title": "CGPA Ranges vs Number of students for MSc Mathematics",
                            "annotations": []}]),
                dict(label="MSc Physics",
                     method="update",
                     args=[{"visible": [False, False, False, False, False, False, False, False, False, False, False, False, True]},
                           {"title": "CGPA Ranges vs Number of students for MSc Physics",
                            "annotations": []}]),
            ]),
        )
    ])

    # Final Touches
    fig.update_layout(title="CGPA Ranges vs Number of students for All Branches",xaxis_title='CG Ranges', yaxis_title='Number of students')
    fig.update_xaxes(categoryorder='array', categoryarray=["(4, 5]", "(5, 6]", "(6, 7]", "(7, 8]","(8, 9]","(9, 10]"])
    fig.update_traces(hovertemplate =
    'Number of students: %{y}'+'<br>CG range: %{x}<br><extra></extra>')
    fig.show()

cghistogram(df)


In [8]:
# Bar plot to show number of PS to TS conversions branchwise
def PStoTS(df):

    # Create new dataframe with number of conversions grouped by branch
    ts = pd.DataFrame(df[df["ConvertedToTS"]==True].groupby("Branch").count()["ID"]).reset_index()

    # Create bar plot, update hover template and title
    fig = px.bar(ts,x="Branch",y="ID", text_auto=True)
    fig.update_layout(title = 'Number of PS to TS convertions in 2018 batch (Branch Wise)',xaxis_title="Branch",yaxis_title="Number of students")
    fig.update_traces(hovertemplate =
        'Number of students: %{y}'+
        '<br>Branch: %{x}<br>')
    fig.show()
PStoTS(df)

In [9]:
# Function to plot departmentwise number of course withdrawls
def withdrawl():   
    ws2021_2022 = pd.read_excel('./Ws 2021-2022.xlsx')
    ws2022_2023 = pd.read_excel('./Ws 2022-2023.xlsx')
    # merge all rows of ws2021_2022 and ws2022_2023, adding a column "dropping year" in a new dataframe withdraw_df
    ws2022_2023['Dropping Year'] = 2022
    ws2021_2022['Dropping Year'] = 2021
    withdraw_df = pd.concat([ws2021_2022, ws2022_2023], ignore_index=True)
    # withdraw_df.reset_index()
    
    withdraw_df.head(8)
    # plotly pie chart using Subject column, plot shows number of students who dropped out of each subject
    subject_count = withdraw_df['Subject'].value_counts() # number of students who dropped out of each subject
    subject_count = subject_count.reset_index()
    subject_count.columns = ['Subject', 'Count']
    subject_count["Department"] = subject_count["Subject"]
    subject_count = subject_count.drop(['Subject'], axis=1)
    # make Department the first column
    subject_count = subject_count[['Department', 'Count']]
    # print all unique Departments
    print("Unique Departments: " + str(subject_count['Department'].unique()))
    
    # Group the similar branches into departments
    def group_departments(department):
        if department in ['HSS', 'GS']:
            return 'Humanities'
        elif department in ['CS', 'IS']:
            return 'CS-IS'
        elif department in ['ECON', 'FIN', 'MGTS']:
            return 'Economics'
        elif department in ['EEE', 'ECE', 'INSTR']:
            return 'Electronics'
        elif department in ['MF', 'ME', 'AN']:
            return 'Mechanical'
        elif department in ['BIO', 'BIOT']:
            return 'Biology'
        elif department in ['PHY']:
            return 'Physics'
        elif department in ['MATH']:
            return 'Mathematics'
        elif department in ['CE']:
            return 'Civil'
        elif department in ['CHE']:
            return 'Chemical'
        elif department in ['CHEM']:
            return 'Chemistry'
        elif department in ['PHA']:
            return 'Pharmacy'
        else:
            return department
    
    subject_count['Department'] = subject_count['Department'].apply(group_departments)
    subject_count = subject_count.groupby(['Department'])['Count'].sum() #.sum is used to group by department and sum the count of students who dropped out of each subject
    subject_count.reset_index()
    fig = px.pie(subject_count, values='Count', names=subject_count.index, title='Distribution of number withdrawls per department 2021-2022 and 2022-2023')
    fig.show()
withdrawl()   

Unique Departments: ['HSS' 'BITS' 'CS' 'ECON' 'EEE' 'GS' 'FIN' 'PHY' 'MATH' 'ME' 'MF' 'ECE'
 'CE' 'CHE' 'MGTS' 'PHA' 'BIO' 'BIOT' 'IS' 'AN' 'CHEM' 'INSTR']


In [10]:
def bitsatVScgScatter(df):    
    scatter_df = df[["Final_ID", "Name", "CGPA", "BITSAT Score", "Branch"]]
    # Scatter plot for CGPA vs BITSAT Score
    fig = px.scatter(scatter_df, x="BITSAT Score", y="CGPA", color="Branch", hover_data=['Name'])

    # Add range slider to the scatter plot, slider on X-axis. Dont display plot points on the slider
    fig.update_layout(
        xaxis=dict(
            rangeslider=dict(
                visible=True
            ),
            type="linear",
        ),
        autosize=False,
        width=1400,
        height=600,
    )


    fig.show()
bitsatVScgScatter(df)    

In [11]:
def cgpa_per_board(df):
    List = df['Board'].value_counts()
    # Histogram to plot average CGPA per board
    fig = px.histogram(df, orientation="h", histfunc="avg", x="CGPA", y="Board",barmode='group', text_auto=True)
    fig.update_traces(hovertemplate =
        '{count}')
    fig.update_layout(title="Average CGPA per Board", xaxis_title='Average CGPA', yaxis_title='Board')
    fig.show()
cgpa_per_board(df)